# Parallel Programming in Scala 

## Instructors: Viktor Kincak and Aleksander Prokopek

> # Week 2: Basic Task-Parallel Algorithms

**Author:** [Ehsan M. Kermani](https://ca.linkedin.com/in/ehsanmkermani)

Codes are available [here](https://github.com/axel22/parprog-snippets/tree/master/src/main/scala/lectures/algorithms)

In [1]:
load.ivy("com.storm-enroute" %% "scalameter-core" % "0.6")

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT from sonatype-snapshots, using Sun Oct 25 12:00:40 PDT 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.10.5;0.3.1-SNAPSHOT from sonatype-snapshots, using Wed Oct 21 02:44:30 PDT 2015

	Choosing sonatype-snapshots for com.github.alexarchambault#ammonite-api_2.10.5;0.3.1-SNAPSHOT

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-api_2.10;0.2.0-SNAPSHOT from sonatype-snapshots, using Wed Oct 21 08:05:05 PDT 2015

	Choosing sonatype-snapshots for com.github.alexarchambault.jupyter#jupyter-api_2.10;0.2.0-SNAPSHOT



In [2]:
import org.scalameter._

import org.scalameter._

In [3]:
/**
  Taken from [[https://github.com/axel22/parprog-snippets]]
*/

import java.util.concurrent._
import scala.util.DynamicVariable

object common {

  val forkJoinPool = new ForkJoinPool

  abstract class TaskScheduler {
    def schedule[T](body: => T): ForkJoinTask[T]
    def parallel[A, B](taskA: => A, taskB: => B): (A, B) = {
      val right = task {
        taskB
      }
      val left = taskA
      (left, right.join())
    }
  }

  class DefaultTaskScheduler extends TaskScheduler {
    def schedule[T](body: => T): ForkJoinTask[T] = {
      val t = new RecursiveTask[T] {
        def compute = body
      }
      Thread.currentThread match {
        case wt: ForkJoinWorkerThread =>
          t.fork()
        case _ =>
          forkJoinPool.execute(t)
      }
      t
    }
  }

  val scheduler =
    new DynamicVariable[TaskScheduler](new DefaultTaskScheduler)

  def task[T](body: => T): ForkJoinTask[T] = {
    scheduler.value.schedule(body)
  }

  def parallel[A, B](taskA: => A, taskB: => B): (A, B) = {
    scheduler.value.parallel(taskA, taskB)
  }

  def parallel[A, B, C, D](taskA: => A, taskB: => B, taskC: => C, taskD: => D): (A, B, C, D) = {
    val ta = task { taskA }
    val tb = task { taskB }
    val tc = task { taskC }
    val td = taskD
    (ta.join(), tb.join(), tc.join(), td)
  }
}


import java.util.concurrent._
import scala.util.DynamicVariable
defined object common

In [5]:
/**
  Taken from [[https://github.com/axel22/parprog-snippets]]
*/

import common._

object MergeSort {
  // a bit of reflection to access the private sort1 method, which takes an offset and an argument
  private val sort1 = {
    val method = scala.util.Sorting.getClass.getDeclaredMethod("sort1", classOf[Array[Int]], classOf[Int], classOf[Int])
    method.setAccessible(true)
    (xs: Array[Int], offset: Int, len: Int) => {
      method.invoke(scala.util.Sorting, xs, offset.asInstanceOf[AnyRef], len.asInstanceOf[AnyRef])
    }
  }

  def quickSort(xs: Array[Int], offset: Int, length: Int): Unit = {
    sort1(xs, offset, length)
  }

  @volatile var dummy: AnyRef = null

  def parMergeSort(xs: Array[Int], maxDepth: Int): Unit = {
    // 1) Allocate a helper array.
    // This step is a bottleneck, and takes:
    // - ~76x less time than a full quickSort without GCs (best time)
    // - ~46x less time than a full quickSort with GCs (average time)
    // Therefore:
    // - there is a almost no performance gain in executing allocation concurrently to the sort
    // - doing so would immensely complicate the algorithm
    val ys = new Array[Int](xs.length)
    dummy = ys
    
    // 2) Sort the elements.
    // The merge step has to do some copying, and is the main performance bottleneck of the algorithm.
    // This is due to the final merge call, which is a completely sequential pass.
    def merge(src: Array[Int], dst: Array[Int], from: Int, mid: Int, until: Int) {
      var left = from
      var right = mid
      var i = from
      while (left < mid && right < until) {
        while (left < mid && src(left) <= src(right)) {
          dst(i) = src(left)
          i += 1
          left += 1
        }
        while (right < until && src(right) <= src(left)) {
          dst(i) = src(right)
          i += 1
          right += 1
        }
      }
      while (left < mid) {
        dst(i) = src(left)
        i += 1
        left += 1
      }
      while (right < mid) {
        dst(i) = src(right)
        i += 1
        right += 1
      }
    }
    // Without the merge step, the sort phase parallelizes almost linearly.
    // This is because the memory pressure is much lower than during copying in the third step.
    def sort(from: Int, until: Int, depth: Int): Unit = {
      if (depth == maxDepth) {
        quickSort(xs, from, until - from)
      } else {
        val mid = (from + until) / 2
        val right = task {
          sort(mid, until, depth + 1)
        }
        sort(from, mid, depth + 1)
        right.join()

        val flip = (maxDepth - depth) % 2 == 0
        val src = if (flip) ys else xs
        val dst = if (flip) xs else ys
        merge(src, dst, from, mid, until)
      }
    }
    sort(0, xs.length, 0)

    // 3) In parallel, copy the elements back into the source array.
    // Executed sequentially, this step takes:
    // - ~23x less time than a full quickSort without GCs (best time)
    // - ~16x less time than a full quickSort with GCs (average time)
    // There is a small potential gain in parallelizing copying.
    // However, most Intel processors have a dual-channel memory controller,
    // so parallel copying has very small performance benefits.
    def copy(src: Array[Int], target: Array[Int], from: Int, until: Int, depth: Int): Unit = {
      if (depth == maxDepth) {
        Array.copy(src, from, target, from, until - from)
      } else {
        val mid = from + ((until - from) / 2)
        val right = task {
          copy(src, target, mid, until, depth + 1)
        }
        copy(src, target, from, mid, depth + 1)
        right.join()
      }
    }
    copy(ys, xs, 0, xs.length, 0)
  }

  val standardConfig = config(
    Key.exec.minWarmupRuns -> 20,
    Key.exec.maxWarmupRuns -> 60,
    Key.exec.benchRuns -> 60,
    Key.verbose -> true
  ) withWarmer(new Warmer.Default)

  def initialize(xs: Array[Int]) {
    var i = 0
    while (i < xs.length) {
      xs(i) = i % 100
      i += 1
    }
  }

  def main(args: Array[String]) {
    val length = 10000000
    val maxDepth = 7
    val xs = new Array[Int](length)
    val seqtime = standardConfig setUp {
      _ => initialize(xs)
    } measure {
      quickSort(xs, 0, xs.length)
    }
    println(s"sequential sum time: $seqtime ms")

    val partime = standardConfig setUp {
      _ => initialize(xs)
    } measure {
      parMergeSort(xs, maxDepth)
    }
    println(s"fork/join time: $partime ms")
    println(s"speedup: ${seqtime / partime}")
  }

}

import common._
defined object MergeSort

In [6]:
MergeSort.main(Array("start"))

Starting warmup.
0. warmup run running time: 203.452077 (covNoGC: NaN, covGC: NaN)
1. warmup run running time: 152.031795 (covNoGC: 0.2046, covGC: 0.2046)
2. warmup run running time: 153.511797 (covNoGC: 0.1725, covGC: 0.1725)
3. warmup run running time: 152.972406 (covNoGC: 0.1530, covGC: 0.1530)
4. warmup run running time: 152.600775 (covNoGC: 0.1391, covGC: 0.1391)
5. warmup run running time: 153.380845 (covNoGC: 0.1280, covGC: 0.1280)
6. warmup run running time: 152.875741 (covNoGC: 0.1194, covGC: 0.1194)
7. warmup run running time: 152.836866 (covNoGC: 0.1123, covGC: 0.1123)
8. warmup run running time: 152.854421 (covNoGC: 0.1064, covGC: 0.1064)
9. warmup run running time: 153.077365 (covNoGC: 0.1012, covGC: 0.1012)
10. warmup run running time: 153.389643 (covNoGC: 0.0967, covGC: 0.0967)
11. warmup run running time: 153.32059 (covNoGC: 0.0927, covGC: 0.0927)
12. warmup run running time: 153.460458 (covNoGC: 0.0892, covGC: 0.0892)
13. warmup run running time: 154.060096 (covNoGC: 0

## Parallel Collections

### Choice of Data Structure:

* (Cons) **List** is not suitable because of linear time search and concatenation
* *Alternatives*: **Array** and **Tree**

```scala
// sequential map for Array
def mapASegSeq[A, B](inp: Array[A], left: Int, right: Int, f: A => B, out: Array[B]) = {
    var i = left
    while(i < right) {
        out(i) = f(inp(i))
        i += 1
    }
}

// paralle map for Array
def mapASegPar[A, B](inp: Array[A], left: Int, right: Int, f: A => B, out: Array[B] = {
    if (right - left < threshold)
        // use sequential for effciency
        mapASegSeq(inp, left, right, f, out)
    else {
        val mid = left + (right - left) / 2
        parallel(mapASegPar(inp, left, mid, f, out), mapASegPar(inp, mid, right, f, out)) 
    }
}
```

In fact, parallelization pays off but manually reimplementing all these operations instead using higher-order functions doesn't and is not elegant!

## Array vs. (Immutable) Tree:

### Array:

* *Advantages*:

    1. Random access to elements on shared memory can share array
    2. Memory locality


* *Diadvantages*:

    1. Must ensure parallel tasks write to disjoint parts
    2. Expensive concatenation

### Immutable Tree:

* *Advantages*:

    1. Purely functional, create new one and keep old one
    2. Disjointness of parallel tasks happens readily
    3. Efficient to combine two Trees

* *Disadvantages*:

    1. High memory location overhead
    2. Bad memory locality

## Parallel Fold (Reduce):

* Need *associative operation* i.e. paratheses (orders of evaluations) don't matter! (Addition is associative while subtraction is not).

* Build trees for expressions where leaves are values and nodes are the operations.

```scala
sealed abstract class Tree[A]
case class Leaf[A](value: A) extends Tree[A]
case class Node[A](left: Tree[A], right: Tree[A]) extends Tree[A]
```
### Sequential Reduce for Tree:

```scala
def reduce[A](t: Tree[A], f: (A, A) => A): A = t match {
    case Leaf(v) => v
    case Node(l, r) => f(reduce(l, f), reduce(r, f))
}
```

### Parallel Reduce for Tree:

```scala
def reduceTreePar[A](t: Tree[A], f: (A, A) => A): A = t match {
    case Leaf(v) => f(v)
    case Node(l, r) => {
        val (lv, rv) = parallel(reduceTreePar(l, f), reduceTreePar(r, f))
        f(lv, rv)
    }
}
```

NOTE: [Tree ratotions](https://en.wikipedia.org/wiki/Tree_rotation) are simply parantheses jaxtapositions.

### Parallel Reduce for Array:

* Assuming associative operation, we can choose *any* tree preserving the order of elements of the original Array (collection), such as balanced tree, etc.

* Advantage: replace `Node` constructor with our associative operation $f,$ instead of directly building the tree
* For small Arrays, use sequential reduce.

```scala
def reduceSegPar[A](inp: Array[A], left: A, right: A, f: (A, A) => A): A = {
    if (right - left < threshold){
        var res = inp(left)
        var i = left+1
        while (i < right) {
            res = f(res, i+1)
            i += 1
        }
    }
    else {
        val mid = left + (right - left) / 2
        val (a1, a2) = parallel(reduceSegPar(inp, left, mid, f), reduceSegPar(inp, mid, right, f))
        f(a1, a2)
    }
}

def reducePar[A](inp: Array[A], f: (A, A) => A): A = reduceSegPar[A](inp, 0, inp.length, f)
```

NOTE: Even in sequential case, we need accosiativity to guarantee that the results of `reduce, reduceLeft` and `reduceRight` agree.

## Associative vs. Commutative:

* Neither implies the other!
* Floating point addition and multiplication are commutative but not accosiative (machine percision flaw!)

### Make an operation commutative (if there's ordering on the input)
```scala
def f(x: A, y: A) = if (less(y, x)) g(y, x) else g(x, y)
```

### Create accosiative function out of two accosiative functions:

Let $f_1: (A_1, A_1) => A_1, f_2: (A_2, A_2) => A_2$ be accosiative, then the function $f: \left((A_1, A_2),(A_1, A_2)\right) => (A_1, A_2)$ mapping 

$$f \left((x_1, x_2),(y_1, y_2) \right) = (f_1(x_1, y_1), f_2(x_2, y_2))$$ 

is associative.

NOTE: So map-reduce is possible! for example, computing average `val res = reduce(map(collection, (x: Int) => (x, 1)), _+_)` then `val average = res._1/res._2`

## Parallel Scan (Prefix Sum):

### Sequential Scan:

* $\text{List}(a_1, \cdots , a_n).\text{scanLeft}(a_0)(f) = \text{List}(b_0, \cdots, b_n)$

where $f$ is an *associative* function, $b_0 = a_0$ and $b_i = f(b_{i-1}, a_i).$

```scala
scanLeftSeq[A](inp: Array[A], a0: A, f: (A, A) => A, out: Array[A]): Unit = {
    out(0) = a0
    var i = 0
    var a = a0
    while(i < inp.length) {
        a = f(a, inp(i))
        i += 1
        out(i) = a
    }
}
```

### Parallel Scan:

For simplicity, assume our initial input is given as a *tree*. So for input and result, we have

```scala
sealed abstract class Tree[A]
case class Leaf[A](a: A) extends Tree[A]
case class Node[A](l: Tree[A], r: Tree[A]) extends Tree[A]

sealed abstract class TreeRes[A] {
    val res: A
}
case class LeafRes[A](override val res: A) extends TreeRes[A] // res is now a field as well as a constructor
class NodeRes[A](l: TreeRes[A], override val res: A, r: TreeRes[A]) extends TreeRes[A]
```
* **Step 1:** *Going up* in parallel i.e. parallel reduce (upsweep)

```scala
// bottom up 
def unsweep[A](t: Tree[A], f: (A, A) => A): TreeRes[A] = t match { // using case class in our advantage
    case Leaf(v) => LeafRes(f(v))
    case Node(l, r) => {
        val (tL, tR) = parallel(unsweep(l, f), unsweep(r, f))
        reduceRes(tL, f(tL.res, tR.res), tR)
    }
}
```
* **Step 2:** *Going down* in parallel

```scala
// 'a0' is the reduce of all elements left to tree 't'
def downsweep[A](t: TreeRes[A], a0: A, f: (A, A) => A): Tree[A] = t match {
    case LeafRes(a) => Leaf(f(a0, a))
    case NodeRes(l, _, r) => {
        val (tL, tR) = parallel(downsweep(l, a0, f), downsweep(r, f(a0, l.res), f))
        Node(tL, tR)
    }
}

```

Therefore,

```scala
def scanLeftPar[A](t: Tree[A], a0: A, f: (A, A) => A): Tree[A] = {
    val tRes = upsweep(t, f) // tree of results
    val scl = downsweep(tRes, a0, f)
    prepend(a0, scl)
}

// not worry about balancing
def prepend[A](x: A, t: Tree[A]): Tree[A] = t match {
    case Leaf(v) => Node(Leaf(x), Leaf(v))
    case Node(l, r) => Node(prepend(x, l), r)
}
```
* For an approximately balanced tree the parallel scan is of $O(\log n)$ (with enough resources) where $n$ is the number of elements in the initial collection (above is number of leaves plus nodes).

* For parallel scan on an *Array*, the process will be very similar (need to store indices/pointer to the begin and end of an array segment when computing intermediate results) except, we perform the computation sequentially when our array is small.